# Pipeline Data understanding and preparation

## Libraries importeren

In [ ]:
# imported libraries
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## DataFrame inladen

Laad het DataFrame in met de kolommen die gebruikt worden.

In [ ]:
cols_to_use = [
    '#stm_sap_meldnr',
    'stm_geo_mld',
    'stm_prioriteit',
    'stm_geo_gst',
    'stm_oorz_groep',
    'stm_oorz_code',
    'stm_fh_ddt',
    'stm_contractgeb_mld',
    'stm_techn_mld',
    'stm_techn_gst',
    'stm_aanntpl_dd',
    'stm_aanntpl_tijd',
    'stm_progfh_in_duur',
    'stm_fh_dd',
    'stm_fh_tijd',
    'stm_fh_duur',
]

df = pd.read_csv("data/sap_storing_data_hu_project.csv", index_col=0, usecols=cols_to_use, engine='pyarrow')
df.head()

## Prepareren Target variabele

De gekozen target variabele is de tijd (in minuten) vanaf het moment dat de aannemer ter plaatse is tot functie herstel.

In [ ]:
# alle relevante kolommen veranderen naar datetimes
df['stm_aanntpl_tijd'] = pd.to_datetime(df['stm_aanntpl_tijd'], format='%H:%M:%S', errors='coerce')
df['stm_aanntpl_dd'] = pd.to_datetime(df['stm_aanntpl_dd'], format='%d/%m/%Y', errors='coerce')
df['stm_fh_ddt'] = pd.to_datetime(df['stm_fh_ddt'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

df[['stm_aanntpl_tijd', 'stm_aanntpl_dd', 'stm_fh_ddt']].info()

In [ ]:
df['stm_aanntpl_tijd'] = df['stm_aanntpl_tijd'].astype('str')
df['stm_aanntpl_dd'] = df['stm_aanntpl_dd'].astype('str')
# de twee kolommen samen voegen om er 1 datetime feature van te maken
df["aanntpl_ddt"] = df["stm_aanntpl_dd"] + " " + df["stm_aanntpl_tijd"].apply(lambda x: x.split(' ')[-1])
df['aanntpl_ddt'] = pd.to_datetime(df['aanntpl_ddt'], format='%Y-%m-%d %H:%M:%S', errors='coerce') 
print(df['aanntpl_ddt'].isna().sum())
df = df.dropna(subset=['aanntpl_ddt'])
df['aanntpl_ddt'].sample(30)

In [ ]:
# bereken de duur van aannemer ter plaatse tot functieherstel in minuten
df['anm_tot_fh'] = df['stm_fh_ddt'] - df['aanntpl_ddt']
df['anm_tot_fh'] = df['anm_tot_fh'].apply(lambda x: x.seconds/60 + x.days * (24*60))
df = df.dropna(subset=['anm_tot_fh'])
df['anm_tot_fh'].sample(30)


In [ ]:
# verwijder alle negatieve waardes, omdat dit er niet zo veel zijn
df = df[df['anm_tot_fh'] >= 0]

In [ ]:
df['anm_tot_fh'].plot.box()

In [ ]:
# Houdt alleen waardes vanaf 5 min tot en met 8 uur (480 min), andere waardes mogen genegeerd worden van de opdrachtgever.
df = df[(df['anm_tot_fh'] >= 5) & (df['anm_tot_fh'] <= 480)]
len(df)

In [ ]:
df['anm_tot_fh'].plot.box()

# Baseline
Voor de baseline is er gekozen om het gemiddelde te nemen van de target variable.

In [ ]:
import baseline
baseline_rmse, baseline_r2 = baseline.calculate_baseline(df)

print('Baseline RMSE: ', baseline_rmse)
print('Baseline R2: ', baseline_r2)

# Resultaten
Hierboven is te zien dat de baseline een R2 score heeft van 0.0 en de MSE is ongeveer 5876.64. Dit is niet een goede score, maar goed genoeg voor een baseline.